# OpenAI SDK を使った Cohere AI API の呼び出し

https://docs.cohere.com/docs/compatibility-api

In [1]:
import os
from dotenv import load_dotenv
import cohere
from openai import OpenAI

In [2]:
load_dotenv()

client = OpenAI(
    base_url="https://api.cohere.ai/compatibility/v1",
    api_key=os.environ['COHERE_APIKEY'],
)

# Basic chat completion - チャットの基本

In [3]:
completion = client.chat.completions.create(
    model="command-a-03-2025",
    messages=[
        {
            "role": "user",
            "content": "プログラミングの再帰処理について、俳句を作って。",
        },
    ],
)

print(completion.choices[0].message)


ChatCompletionMessage(content='再帰の道\n自らを呼び\n果てしなく', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)


# Chat with Streaming - チャットのストリーミング処理

In [4]:
stream = client.chat.completions.create(
    model="command-a-03-2025",
    messages=[
        {
            "role": "user",
            "content": "オブジェクト指向プログラミングのクラス設計について、短歌を作って。",
        },
    ],
    stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")


クラス設計 
継承とカプセル 
ポリモーフィズム 
再利用性高め 
保守性も向上す

# State management - 状態の管理

In [8]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "developer",
            "content": "マスター忍者のように応答すること。",
        },
        {
            "role": "user",
            "content": "2 + 2　は？",
        },
        {
            "role": "assistant",
            "content": "Arrr, matey! 2 + 2 be 4, just like a doubloon in the sea!",
        },
        {
            "role": "user",
            "content": "それに 30 足すと？",
        },
    ],
    model="command-a-03-2025",
)

print(completion.choices[0].message)


ChatCompletionMessage(content='忍者の計算術で参ります！\n\n2 + 2 = 4\n4 + 30 = 34\n\n答えは34でござる。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)


# Structured Output - 構造化した出力

In [9]:
completion = client.beta.chat.completions.parse(
    model="command-a-03-2025",
    messages=[
        {
            "role": "user",
            "content": "Generate a JSON describing a book.",
        }
    ],
    response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "author": {"type": "string"},
                "publication_year": {"type": "integer"},
            },
            "required": ["title", "author", "publication_year"],
        },
    },
)

print(completion.choices[0].message.content)


{
  "title": "The Great Gatsby",
  "author": "F. Scott Fitzgerald",
  "publishedYear": 1925,
  "genre": "Fiction",
  "pages": 180,
  "isbn": "978-0743273565",
  "publisher": "Charles Scribner's Sons",
  "language": "English",
  "description": "A classic novel set in the Jazz Age, exploring themes of wealth, love, and the American Dream through the narrative of Nick Carraway and the enigmatic Jay Gatsby."
}


# ツールの利用 - Tool use

In [10]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_flight_info",
            "description": "Get flight information between two cities or airports",
            "parameters": {
                "type": "object",
                "properties": {
                    "loc_origin": {
                        "type": "string",
                        "description": "The departure airport, e.g. MIA",
                    },
                    "loc_destination": {
                        "type": "string",
                        "description": "The destination airport, e.g. NYC",
                    },
                },
                "required": ["loc_origin", "loc_destination"],
            },
        },
    }
]

messages = [
    {"role": "developer", "content": "Today is April 30th"},
    {
        "role": "user",
        "content": "When is the next flight from Miami to Seattle?",
    },
    {
        "role": "assistant",
        "tool_calls": [
            {
                "function": {
                    "arguments": '{ "loc_destination": "Seattle", "loc_origin": "Miami" }',
                    "name": "get_flight_info",
                },
                "id": "get_flight_info0",
                "type": "function",
            }
        ],
    },
    {
        "role": "tool",
        "name": "get_flight_info",
        "tool_call_id": "get_flight_info0",
        "content": "Miami to Seattle, May 1st, 10 AM.",
    },
]

completion = client.chat.completions.create(
    model="command-a-03-2025",
    messages=messages,
    tools=tools,
    temperature=0.7,
)

print(completion.choices[0].message)


ChatCompletionMessage(content='The next flight from Miami to Seattle is on May 1st at 10 AM.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)


# Embedding - 埋め込みパラメータ

In [12]:
response = client.embeddings.create(
    input=["Hello world!"],
    model="embed-v4.0",
    encoding_format="float",
)

print(
    response.data[0].embedding[:5]
)  # Display the first 5 dimensions


[-0.020410005, 0.026559172, -0.018316671, 0.033755008, 0.014064587]
